<a href="https://colab.research.google.com/github/talhadar1/real_estate_price_forecasting/blob/master/real_estat_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
import os
os.getcwd()
# !pip install -U pandas-profiling



'/content'

In [0]:
# #Adding more RAM to the COLAB machine:
# #https://towardsdatascience.com/upgrade-your-memory-on-google-colab-for-free-1b8b18e8791d
# a = []
# while(1):
#     a.append('1')

# **Import libraries and files:**

In [0]:
import pandas as pd
import pandas_profiling
from pandas_profiling import ProfileReport
from sklearn.model_selection import train_test_split
import numpy as np
folder_path = '/content/drive/My Drive/seminar_data/'
file_name = 'TransactionsXLSsince2008.xlsx'

# **import data:**

In [0]:
original = pd.read_excel(folder_path + file_name)

In [0]:
original.head()

,gush_helka,tat_helka,date_transaction,price_sheqel,city_hebrew,rechov,year_of_construction,bayit,floor,shetachneto,type,floors,dirotbnyn,dira,year,month,day,soceco
0,007621-0467,060-00,2019-07-25,2585000.0,כפר סבא,לוונברג,2011.0,9.0,1.0,112.0,דירה בבית קומות,6.0,19.0,4.0,2019.0,7.0,25.0,0.139017
1,007074-0022,060-00,2019-07-25,1940000.0,תל אביב -יפו,נחל הבשור,1970.0,1.0,11.0,72.0,דירה בבית קומות,14.0,72.0,0.0,2019.0,7.0,25.0,-0.499218
2,012612-0008,044-00,2019-07-25,1400000.0,אור עקיבא,הכרמל,2005.0,13.0,1.0,85.0,דירה בבית קומות,4.0,11.0,2.0,2019.0,7.0,25.0,-0.759605
3,006435-0284,023-00,2019-07-24,1800000.0,כפר סבא,נורדאו,1998.0,2.0,2.0,109.0,דירה בבית קומות,7.0,29.0,4.0,2019.0,7.0,24.0,0.294809
4,007960-0031,028-00,2019-07-24,1220000.0,נתניה,קצנלסון יצחק,1970.0,13.0,7.0,62.0,דירה בבית קומות,8.0,32.0,28.0,2019.0,7.0,24.0,-0.647234


In [0]:
profile_report = original.profile_report(html={'style': {'full_width': True}})
profile.to_file(outputfile = folder_path + 'original_data.html')

# ProfileReport(original,style={'full_width':True})
# ProfileReport(original)
# original.profile_report(style={'full_width':True})



1.   Note that the transaction date appears as both date_transaction and split date year, month and day. The full date does not give us good data for days and months so we will lower it. And leave the three properties of the second option.
2.  


---



validate the dimensions of the dataframe:

In [0]:
# dataframe.size - number of cells
size = df.size 
  
# dataframe.shape - shape[0]=number of rows  | shape[1]=number of columns
shape = df.shape 

# printing size and shape 
print("number of cells = {}\nShape ={}\#ROWS x #COLUMNS = {}". format(size, shape, shape[0]*shape[1])) 

In [0]:
profile.to_file(outputfile = folder_path + 'original_data.html')
df = original.drop(['rechov', 'date_transaction', 'city_hebrew', 'tat_helka', 'bayit', 'dira'], axis=1).dropna(how='all')


number of cells = 7975140
Shape =(664595, 12)
Shape[0] x Shape[1] = 7975140



1.   Note that the transaction date appears as both date_transaction and split date year, month and day. The full date does not give us good data for days and months so we will lower it. And leave the three properties of the second option.
2.  


---



In [0]:
df = df[df['year']>1900]
df['floors'] = df['floors'].fillna(5)
df['dirotbnyn'] = df['dirotbnyn'].fillna(18)
df['soceco'] = df['soceco'].fillna(0.16788)


,gush_helka,price_sheqel,year_of_construction,floor,shetachneto,type,floors,dirotbnyn,year,month,day,soceco
0,007621-0467,2585000.0,2011.0,1.0,112.0,דירה בבית קומות,6.0,19.0,2019.0,7.0,25.0,0.139017
1,007074-0022,1940000.0,1970.0,11.0,72.0,דירה בבית קומות,14.0,72.0,2019.0,7.0,25.0,-0.499218
2,012612-0008,1400000.0,2005.0,1.0,85.0,דירה בבית קומות,4.0,11.0,2019.0,7.0,25.0,-0.759605
3,006435-0284,1800000.0,1998.0,2.0,109.0,דירה בבית קומות,7.0,29.0,2019.0,7.0,24.0,0.294809
4,007960-0031,1220000.0,1970.0,7.0,62.0,דירה בבית קומות,8.0,32.0,2019.0,7.0,24.0,-0.647234


In [0]:
df_temp = df.sample(frac=.7)
del df    
df = df_temp
del df_temp
df['gush'] = df['gush_helka'].apply(lambda x: x.split('-')[0])

In [0]:
df = df.drop('gush_helka', axis=1)
df.head()

,price_sheqel,year_of_construction,floor,shetachneto,type,floors,dirotbnyn,year,month,day,soceco,gush
358698,1100000.0,1964.0,2.0,54.0,דירה בבית קומות,4.0,10.0,2013.0,2.0,27.0,-1.046986,006122
334026,2391595.0,2013.0,13.0,134.0,דירה בבית קומות,17.0,54.0,2013.0,7.0,23.0,0.439560,003946
610031,1640000.0,2009.0,2.0,117.0,דירה בבית קומות,8.0,33.0,2009.0,9.0,13.0,1.781149,006715
457860,650000.0,1970.0,0.0,85.0,דירה בבית קומות,5.0,18.0,2011.0,9.0,21.0,-0.677741,010570
560735,1349999.0,1997.0,1.0,160.0,קוטג' חד-משפחתי,1.0,18.0,2010.0,5.0,6.0,0.167880,010562


In [0]:
profile = ProfileReport(df)
profile.to_file(outputfile="/content/drive/My Drive/Colab Notebooks/cleaned_data.html")

/usr/local/lib/python3.6/dist-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


In [0]:
df = pd.get_dummies(df.dropna())
df.head()
df_2 = df.drop('soceco', axis=1)

,price_sheqel,year_of_construction,floor,shetachneto,floors,dirotbnyn,year,month,day,soceco,type_בית בודד,type_דירה בבית קומות,type_דירת גג,type_דירת גן,type_דירת דופלקס,type_קוטג' דו-משפחתי,type_קוטג' חד-משפחתי,type_קוטג' טורי,gush_000001,gush_000071,gush_000117,gush_000119,gush_000124,gush_000181,gush_000182,gush_000187,gush_000196,gush_000204,gush_000240,gush_000259,gush_000265,gush_000266,gush_000268,gush_000283,gush_000301,gush_000309,gush_000310,gush_000311,gush_000315,gush_000326,...,gush_201047,gush_202000,gush_202009,gush_202011,gush_204003,gush_204004,gush_204005,gush_204006,gush_204007,gush_204011,gush_204012,gush_204015,gush_204017,gush_204021,gush_204025,gush_204026,gush_204028,gush_204101,gush_204301,gush_204304,gush_204503,gush_204551,gush_204701,gush_204702,gush_215001,gush_220000,gush_242002,gush_246001,gush_260000,gush_279001,gush_400175,gush_400403,gush_400495,gush_400505,gush_401006,gush_401009,gush_401022,gush_691800,gush_692400,gush_900009
358698,1100000.0,1964.0,2.0,54.0,4.0,10.0,2013.0,2.0,27.0,-1.046986,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
334026,2391595.0,2013.0,13.0,134.0,17.0,54.0,2013.0,7.0,23.0,0.439560,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
610031,1640000.0,2009.0,2.0,117.0,8.0,33.0,2009.0,9.0,13.0,1.781149,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
457860,650000.0,1970.0,0.0,85.0,5.0,18.0,2011.0,9.0,21.0,-0.677741,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
560735,1349999.0,1997.0,1.0,160.0,1.0,18.0,2010.0,5.0,6.0,0.167880,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


split data to train\test:

In [0]:
train_df, test_df, y_train, y_test = train_test_split(df.drop('price_sheqel', axis=1), df['price_sheqel'], test_size=.2)

In [0]:
#Generating dataframe without the 'soceco' feature
train_df2, test_df2, y_train2, y_test2 = train_df.drop('soceco', axis=1), test_df.drop('soceco', axis=1), y_train, y_test

NameError: ignored

In [0]:
del df
del df_2

### **Random Forest Regressor :**

In [0]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
min_samples_in_a_leaf = 30

In [0]:
# Instantiate model with 100 decision trees
rf = RandomForestRegressor(n_estimators = 100, random_state = 42, verbose=100, min_samples_leaf=min_samples_in_a_leaf, n_jobs=-1)
rf_no_soceco = RandomForestRegressor(n_estimators = 100, random_state = 42, verbose=100, min_samples_leaf=min_samples_in_a_leaf, n_jobs=-1)


####Train the model on training data


In [0]:
#Train the models on training data
rf.fit(train_df, y_train)
rf_no_soceco.fit(train_df2, y_train2)

####Test the model on the the test data

In [0]:
# Use the forest's predict method on the test data
predictions  = rf.predict(test_df)
predictions_no_soceco = rf_no_soceco.predict(test_df2)

# Calculate the absolute errors
errors = abs(predictions - y_test)
# Print out the mean absolute error (mae)
print(f'Mean Absolute Error: {round(np.mean(errors), 2)} shekel')
print(f'Median Absolute Error: {round(np.median(errors), 2)} shekel')

# Calculate the absolute errors
errors = abs(predictions_no_soceco - y_test2)
# Print out the mean absolute error (mae)
print(f'Predictions_no_soceco mean Absolute Error: {round(np.mean(errors), 2)} shekel')
print(f'Predictions_no_soceco median Absolute Error: {round(np.median(errors), 2)} shekel')

####Save model to file:

In [0]:
import joblib
joblib_file =folder_path + 'RandomForestRegressor_model.pkl'
joblib_file =folder_path + 'RandomForestRegressor_model_no_soceco.pkl'
joblib.dump(rf,joblib_file)
joblib.dump(rf_no_soceco,joblib_file_no_soceco)

# test_df.to_csv (folder_path+ r'export_dataframe.csv', index = False, header=True)
# y_test.to_csv  (folder_path + r'export_dataframe.csv', index = False, header=True)
dataframe_row = folder_path + 'dataframe_row.csv'
test_df.head(1).to_csv (dataframe_row, index = False, header=True)


#### Load model from file:

In [0]:
loaded_model = joblib.load(joblib_file)


In [0]:
test_index = 3
input_row = test_df.head(test_index)
input_row = test_df.head(test_index)
real_output = y_test.head(test_index)
print(input_row )
print(real_output)
prediction = loaded_model.predict(input_row)
error = abs(prediction - y_test.head(test_index))
print(f'Prediction Error: {error} shekel')

# prediced_row.head()

In [0]:
test_index = 3
input_row = test_df.iloc[[test_index]]
print (input_row)
real_output = y_test.iloc[[test_index]]
print(input_row )
print(real_output)
prediction = loaded_model.predict(input_row)
error = abs(prediction - y_test.iloc[[test_index]])
print(f'Prediction Error: {error} shekel')
print(error)


In [0]:
importances = list(zip(train_df.columns.to_list(), rf.feature_importances_))
epsilon = 0.0001
count = 0
for col, val in sorted(importances, key=lambda x: x[1], reverse=True):
    if val > epsilon:
      count = count + 1
      print(f'{col}: {val}')
print(count)

In [0]:
# original.dropna()[original['gush_helka'].dropna().str.contains('009126')].sort_values('price_sheqel')

### **Linear Regression:**

In [0]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression()
reg.fit(train_df, y_train)

In [0]:
from sklearn.linear_model import SGDRegressor
reg = SGDRegressor()
reg.fit(train_df, y_train)

In [0]:
# Use the forest's predict method on the test data
predictions = reg.predict(test_df)

In [0]:
# Calculate the absolute errors
errors = abs(predictions - y_test)
# Print out the mean absolute error (mae)
print(f'Mean Absolute Error: {round(np.mean(errors), 2)} shekel')